https://www.tensorflow.org/get_started/get_started

In [ ]:
import tensorflow as tf

# Constants

In [ ]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

In [ ]:
sess = tf.Session()
print(sess.run([node1, node2]))

In [ ]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

# Placeholders

In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # or tf.add(a, b)

In [ ]:
a

In [ ]:
adder_node

In [ ]:
params = {a: 3, b: 4.5}
params

In [ ]:
print(sess.run(adder_node, params))
print(sess.run(adder_node, {a: [1, 3], b: [2,4]}))

Add and triple

In [ ]:
add_and_triple = adder_node * 3
add_and_triple

In [ ]:
print(sess.run(add_and_triple, params))

### Linear Model

In [ ]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
W

In [ ]:
print(sess.run(W)) # doing this without initializing global variable will result in an error

In [ ]:
linear_model = W*x + b
linear_model

init = tf.global_variables_initializer()
init

In [ ]:
print(sess.run(init))

In [ ]:
print(sess.run(W))

In [ ]:
x

In [ ]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

### Evaluate a model

We need:
- a placeholder **y** to evaluate if the model output matches the **target** value
- a **loss function** to evaluate how close the 2 values are to each other

In [ ]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

In [ ]:
# Evaluate
training_params = {
    x: [1, 2, 3, 4],
    y: [0,-1,-2,-3]
}
sess.run(loss, training_params)

#### Fix

In [ ]:
fixW = tf.assign(W, [-1.])
fixW

In [ ]:
W

In [ ]:
fixb = tf.assign(b, [1.])
fixb

In [ ]:
sess.run([fixW, fixb])

In [ ]:
sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

### Training a models

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [ ]:
sess.run(init) # reset W and b to incorrect defaults

In [ ]:
sess.run([W,b])

In [ ]:
training_params

In [ ]:
for i in range(1000):
    sess.run(train, training_params)

In [ ]:
sess.run([W, b])

# tf.estimator

Basic Usage

In [ ]:
import numpy as np
import tensorflow as tf

feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train},
    y_train,
    batch_size=4,
    num_epochs=None,
    shuffle=True
)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train},
    y_train,
    batch_size=4,
    num_epochs=1000,
    shuffle=False
)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval},
    y_eval,
    batch_size=4,
    num_epochs=1000,
    shuffle=False
)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)

print("train metrics: %r" % train_metrics)
print("eval metrics: %r" % eval_metrics)

A Custom model

In [ ]:
import numpy as np
import tensorflow as tf

def model_fn(features, labels, mode):
    # Build a linear model and predict the values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W * features["x"] + b
    
    # Loss Sub-Graph
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                    tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train
    )

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_fn)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train},
    y_train,
    batch_size=4,
    num_epochs=None,
    shuffle=True
)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train},
    y_train,
    batch_size=4,
    num_epochs=1000,
    shuffle=False
)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval},
    y_eval,
    batch_size=4,
    num_epochs=1000,
    shuffle=False
)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)

print("train metrics: %r" % train_metrics)
print("eval metrics: %r" % eval_metrics)